In [1]:
!git clone https://github.com/NVIDIA/apex
!cd apex && pip install -v --disable-pip-version-check --no-cache-dir ./

Cloning into 'apex'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 7939 (delta 5), reused 12 (delta 0), pack-reused 7913
Receiving objects: 100% (7939/7939), 14.02 MiB | 10.98 MiB/s, done.
Resolving deltas: 100% (5406/5406), done.
Created temporary directory: /tmp/pip-ephem-wheel-cache-mr4jpzjq
Created temporary directory: /tmp/pip-req-tracker-qa4kob_b
Created requirements tracker '/tmp/pip-req-tracker-qa4kob_b'
Created temporary directory: /tmp/pip-install-y1trm5bp
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-of7gf6md
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-qa4kob_b'
    Running setup.py (path:/tmp/pip-req-build-of7gf6md/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info


    torch.__version__  = 1.8.1+cu101


    running egg_info
    creating /tmp/pip-req-build-of7gf6md/pip

In [ ]:
import os
from datetime import datetime
import argparse
import torch.multiprocessing as mp
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.distributed as dist
from apex.parallel import DistributedDataParallel as DDP
from apex import amp
from tqdm.notebook import tqdm

class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out


def train(gpu, args):
    rank = args['nr'] * args['gpus'] + gpu
    dist.init_process_group(backend='nccl', init_method='env://', world_size=args['world_size'], rank=rank)
    torch.manual_seed(0)
    model = ConvNet()
    torch.cuda.set_device(gpu)
    model.cuda(gpu)
    batch_size = 100
    criterion = nn.CrossEntropyLoss().cuda(gpu)
    optimizer = torch.optim.SGD(model.parameters(), 1e-4)
    model = nn.parallel.DistributedDataParallel(model, device_ids=[gpu])
    train_dataset = torchvision.datasets.MNIST(root='./data',
                                               train=True,
                                               transform=transforms.ToTensor(),
                                               download=True)
    train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset,
                                                                    num_replicas=args['world_size'],
                                                                    rank=rank)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size,
                                               shuffle=False,
                                               num_workers=0,
                                               pin_memory=True,
                                               sampler=train_sampler)

    start = datetime.now()
    total_step = len(train_loader)
    for epoch in range(args['epochs']):
        for i, (images, labels) in tqdm(enumerate(train_loader)):
            images = images.cuda(non_blocking=True)
            labels = labels.cuda(non_blocking=True)
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if (i + 1) % 100 == 0 and gpu == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch + 1, args['epochs'], i + 1, total_step,
                                                                         loss.item()))
    if gpu == 0:
        print("Training complete in: " + str(datetime.now() - start))

args = {
    'nodes':1,
    'gpus':1,
    'nr':0,
    'epochs':2
}
args['world_size'] = args['gpus'] * args['nodes']
os.environ['MASTER_ADDR'] = '10.57.23.164'
os.environ['MASTER_PORT'] = '8888'

train(2, args)